In [ ]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Colon_and_Rectal_Cancer"
cohort = "GSE46862"

# Input paths
in_trait_dir = "../../input/GEO/Colon_and_Rectal_Cancer"
in_cohort_dir = "../../input/GEO/Colon_and_Rectal_Cancer/GSE46862"

# Output paths
out_data_file = "../../output/preprocess/Colon_and_Rectal_Cancer/GSE46862.csv"
out_gene_data_file = "../../output/preprocess/Colon_and_Rectal_Cancer/gene_data/GSE46862.csv"
out_clinical_data_file = "../../output/preprocess/Colon_and_Rectal_Cancer/clinical_data/GSE46862.csv"
json_path = "../../output/preprocess/Colon_and_Rectal_Cancer/cohort_info.json"


### Step 1: Initial Data Loading

In [ ]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [ ]:
# Analyzing data availability and creating conversion functions

# 1. Gene Expression Data Availability
# Based on the background information, this study used Affymetrix GenChip arrays
# for gene expression profiling, so gene expression data should be available
is_gene_available = True

# 2.1 Data Availability
# From the sample characteristics dictionary:
# - trait (chemoradiation therapy response) is in row 0
# - age is in row 1
# - gender (Sex) is in row 2
trait_row = 0
age_row = 1
gender_row = 2

# 2.2 Data Type Conversion Functions
def convert_trait(value):
    """Convert chemoradiation therapy response to binary (1 for good response, 0 for poor response)"""
    if not value or ':' not in value:
        return None
    
    response = value.split(':', 1)[1].strip()
    # Based on the description, NT and TO are better responses, MI and MO are worse responses
    if response == 'NT' or response == 'TO':
        return 1  # good response
    elif response == 'MI' or response == 'MO':
        return 0  # poor response
    else:
        return None

def convert_age(value):
    """Convert age to continuous numeric value"""
    if not value or ':' not in value:
        return None
    
    try:
        age = int(value.split(':', 1)[1].strip())
        return age
    except (ValueError, TypeError):
        return None

def convert_gender(value):
    """Convert gender to binary (0 for female, 1 for male)"""
    if not value or ':' not in value:
        return None
    
    gender = value.split(':', 1)[1].strip().lower()
    if gender == 'male':
        return 1
    elif gender == 'female':
        return 0
    else:
        return None

# 3. Save Metadata
# Determine trait data availability
is_trait_available = trait_row is not None
# Initial filtering
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # Sample characteristics dictionary from previous step
    sample_chars = {
        0: ['chemoradiation therapy response: MO', 'chemoradiation therapy response: TO', 'chemoradiation therapy response: MI', 'chemoradiation therapy response: NT'],
        1: ['age: 68', 'age: 58', 'age: 66', 'age: 56', 'age: 55', 'age: 50', 'age: 37', 'age: 59', 'age: 46', 'age: 49', 'age: 62', 'age: 65', 'age: 63', 'age: 41', 'age: 33', 'age: 73', 'age: 70', 'age: 69', 'age: 39', 'age: 43', 'age: 48', 'age: 72', 'age: 76', 'age: 40', 'age: 54', 'age: 45', 'age: 71', 'age: 52', 'age: 53', 'age: 67'],
        2: ['Sex: male', 'Sex: female'],
        3: ['tumor stage (uicc-7th): IIIB', 'tumor stage (uicc-7th): 0', 'tumor stage (uicc-7th): I', 'tumor stage (uicc-7th): IIA', 'tumor stage (uicc-7th): IIIA', 'tumor stage (uicc-7th): IIIC', 'tumor stage (uicc-7th): IVA', 'tumor stage (uicc-7th): IV', 'tumor stage (uicc-7th): lll', 'tumor stage (uicc-7th): IIB'],
        4: ['description (operation): LAR', 'description (operation): ULAR', 'description (operation): LAR, TAH', 'description (operation): LAR, ileostomy', "description (operation): Hartmann's procedure", 'description (operation): APR', 'description (operation): APR,TAH,LSO,S6 segmentectomy', 'description (operation): TEM', 'description (operation): L-LAR, Rt PLND, ileostomy', 'description (operation): ISR, colonic pouch, ileostomy', 'description (operation): ISR, coloanal, ileostomy', 'description (operation): R-LAR, ileostomy', 'description (operation): ISR, colonic pouch, Ileostomy', 'description (operation): ISR, colonic pouch, ileostomy, Rt PLND', 'description (operation): ISR, Coloanal, ileostomy', 'description (operation): ELAR', 'description (operation): ISR']
    }
    
    # Create a clinical data DataFrame with the structure expected by geo_select_clinical_features
    # Where rows are characteristics and each column would represent a sample
    clinical_data = pd.DataFrame()
    for key, values in sample_chars.items():
        clinical_data.loc[key] = values
    
    # Extract clinical features
    selected_features = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the extracted features
    preview = preview_df(selected_features)
    print("Preview of selected clinical features:")
    print(preview)
    
    # Save the clinical data
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_features.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical data saved to {out_clinical_data_file}")


### Step 3: Dataset Analysis and Clinical Feature Extraction

In [ ]:
import pandas as pd
import os
import json
import gzip
from typing import Optional, Callable, Dict, Any, List
import numpy as np
import re

# Load the clinical data 
clinical_data_file = os.path.join(in_cohort_dir, "GSE46862_series_matrix.txt.gz")

try:
    # Read the clinical data
    with gzip.open(clinical_data_file, 'rt') as file:
        lines = file.readlines()
    
    # Extract sample characteristics
    sample_char_lines = [line for line in lines if line.startswith('!Sample_characteristics_ch1')]
    
    # Extract sample IDs
    sample_id_line = next((line for line in lines if line.startswith('!Sample_geo_accession')), None)
    if sample_id_line:
        sample_ids = sample_id_line.strip().split('\t')[1:]
    else:
        sample_ids = []
    
    # Create a dictionary to organize sample characteristics
    sample_char_dict = {}
    
    for line in sample_char_lines:
        values = line.strip().split('\t')[1:]
        sample_char_dict[len(sample_char_dict)] = values
    
    # Create a DataFrame from the dictionary
    clinical_data = pd.DataFrame(sample_char_dict, index=sample_ids).T
    
    # Print some sample data to analyze
    print("Sample characteristics dictionary:")
    for key, values in sample_char_dict.items():
        unique_values = set(values)
        print(f"Key {key}: {list(unique_values)[:5]}{' ...' if len(unique_values) > 5 else ''} (Unique values: {len(unique_values)})")
    
    # Extract title and description for background information
    title_line = next((line for line in lines if line.startswith('!Series_title')), None)
    title = title_line.strip().split('!Series_title\t')[1] if title_line else "No title found"
    
    description_lines = [line for line in lines if line.startswith('!Series_summary')]
    description = ' '.join([line.strip().split('!Series_summary\t')[1] for line in description_lines]) if description_lines else "No description found"
    
    print("\nDataset Title:")
    print(title)
    print("\nDataset Description:")
    print(description)
    
    # Check if this is a gene expression dataset
    platform_line = next((line for line in lines if line.startswith('!Series_platform_id')), None)
    platform_id = platform_line.strip().split('!Series_platform_id\t')[1] if platform_line else "Unknown"
    
    print("\nPlatform ID:")
    print(platform_id)
    
except Exception as e:
    print(f"Error reading clinical data: {e}")
    clinical_data = pd.DataFrame()
    sample_char_dict = {}

# 1. Gene Expression Data Availability
# Based on platform info (GPL6244) and description, determine if gene expression data is likely available
is_gene_available = True  # The platform GPL6244 is for gene expression

# 2. Clinical Feature Availability and Data Type Conversion
# Analyze sample characteristics to identify trait, age, and gender information

# 2.1 Trait Availability (Chemoradiation therapy response)
trait_row = 0  # Chemoradiation therapy response is in row 0

# Function to convert trait values
def convert_trait(value):
    if value is None or pd.isna(value):
        return None
    
    # Remove quotes and extract value after colon
    value = str(value).strip('"')
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Convert therapy responses to binary
    value_lower = str(value).lower()
    if 'nt' in value_lower or 'mo' in value_lower:  # Non-responders
        return 0
    elif 'mi' in value_lower or 'to' in value_lower:  # Responders
        return 1
    else:
        return None

# 2.2 Age Availability
age_row = 1  # Age information is in row 1

# Function to convert age values
def convert_age(value):
    if value is None or pd.isna(value):
        return None
    
    # Remove quotes and extract value after colon
    value = str(value).strip('"')
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Try to extract numeric age
    try:
        age_match = re.search(r'(\d+)', str(value))
        if age_match:
            return float(age_match.group(1))
    except:
        pass
    
    return None

# 2.3 Gender Availability
gender_row = 2  # Sex information is in row 2

# Function to convert gender values
def convert_gender(value):
    if value is None or pd.isna(value):
        return None
    
    # Remove quotes and extract value after colon
    value = str(value).strip('"')
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Convert to binary: Female = 0, Male = 1
    value_lower = str(value).lower()
    if 'female' in value_lower or 'f' == value_lower:
        return 0
    elif 'male' in value_lower or 'm' == value_lower:
        return 1
    else:
        return None

# 3. Save Metadata - Initial Filtering
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction (if trait_row is not None)
if trait_row is not None:
    # Extract clinical features
    clinical_features_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the extracted features
    print("\nExtracted Clinical Features Preview:")
    print(preview_df(clinical_features_df))
    
    # Create directory if it doesn't exist
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    
    # Save to CSV
    clinical_features_df.to_csv(out_clinical_data_file)
    print(f"Clinical features saved to {out_clinical_data_file}")


### Step 4: Gene Data Extraction

In [ ]:
# 1. Get the file paths for the SOFT file and matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. First, let's examine the structure of the matrix file to understand its format
import gzip

# Peek at the first few lines of the file to understand its structure
with gzip.open(matrix_file, 'rt') as file:
    # Read first 100 lines to find the header structure
    for i, line in enumerate(file):
        if '!series_matrix_table_begin' in line:
            print(f"Found data marker at line {i}")
            # Read the next line which should be the header
            header_line = next(file)
            print(f"Header line: {header_line.strip()}")
            # And the first data line
            first_data_line = next(file)
            print(f"First data line: {first_data_line.strip()}")
            break
        if i > 100:  # Limit search to first 100 lines
            print("Matrix table marker not found in first 100 lines")
            break

# 3. Now try to get the genetic data with better error handling
try:
    gene_data = get_genetic_data(matrix_file)
    print(gene_data.index[:20])
except KeyError as e:
    print(f"KeyError: {e}")
    
    # Alternative approach: manually extract the data
    print("\nTrying alternative approach to read the gene data:")
    with gzip.open(matrix_file, 'rt') as file:
        # Find the start of the data
        for line in file:
            if '!series_matrix_table_begin' in line:
                break
                
        # Read the headers and data
        import pandas as pd
        df = pd.read_csv(file, sep='\t', index_col=0)
        print(f"Column names: {df.columns[:5]}")
        print(f"First 20 row IDs: {df.index[:20]}")
        gene_data = df


### Step 5: Gene Identifier Review

In [ ]:
# Looking at the identifiers in the gene expression data
# The identifiers appear to be numeric codes (like 7892501, 7892502, etc.)
# These are not standard human gene symbols (which would be like BRCA1, TP53, etc.)
# These are likely probe IDs from a microarray platform that need to be mapped to gene symbols

requires_gene_mapping = True


### Step 6: Gene Annotation

In [ ]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


### Step 7: Gene Identifier Mapping

In [ ]:
# Analyze the format of gene identifiers in both datasets
print("\nIdentifying mapping columns:")
print(f"Gene expression data has identifiers like: {gene_data.index[:5]}")
print(f"Gene annotation data columns: {gene_annotation.columns}")

# The 'ID' column in gene_annotation matches the index in gene_data
# The 'gene_assignment' column contains gene symbols

# Identifying gene symbols in the gene_assignment column
print("\nChecking gene_assignment format:")
sample_gene = gene_annotation['gene_assignment'].iloc[2]
print(f"Sample gene assignment: {sample_gene[:200]}...")

# Extract the mapping between probe IDs and gene symbols
gene_mapping = get_gene_mapping(
    annotation=gene_annotation, 
    prob_col='ID', 
    gene_col='gene_assignment'
)

print("\nMapped genes preview:")
print(gene_mapping.head())

# Apply the gene mapping to convert probe-based expression to gene expression
gene_data = apply_gene_mapping(
    expression_df=gene_data,
    mapping_df=gene_mapping
)

# Preview the gene expression data
print("\nGene expression data after mapping:")
print(f"Shape: {gene_data.shape}")
print(f"First 5 genes: {gene_data.index[:5]}")
print(f"Sample columns: {gene_data.columns[:5]}")

# Save the gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"\nGene expression data saved to {out_gene_data_file}")


### Step 8: Data Normalization and Linking

In [ ]:
# 1. Normalize gene symbols in the gene expression data
gene_data = pd.read_csv(out_gene_data_file, index_col=0)
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Normalized gene data shape: {normalized_gene_data.shape}")
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Retrieve the original clinical data directly from step 2
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
background_info, clinical_df = get_background_and_clinical_data(matrix_file)

# Extract clinical features using the geo_select_clinical_features function as in step 3
clinical_features_df = geo_select_clinical_features(
    clinical_df=clinical_df,
    trait=trait,
    trait_row=0,  # chemoradiation therapy response
    convert_trait=lambda x: 1 if any(resp in str(x).upper() for resp in ["NT", "TO"]) else 0,  # better response = 1
    age_row=1,    # age information
    convert_age=lambda x: int(str(x).split(":", 1)[1].strip()) if ":" in str(x) else None,
    gender_row=2,  # gender information
    convert_gender=lambda x: 0 if "female" in str(x).lower() else 1 if "male" in str(x).lower() else None
)

# Fix column names in gene data to match clinical data format
normalized_gene_data.columns = [col.strip('"') for col in normalized_gene_data.columns]

# 3. Link the clinical and genetic data
linked_data = geo_link_clinical_genetic_data(clinical_features_df, normalized_gene_data)
print(f"Initial linked data shape: {linked_data.shape}")

# 4. Handle missing values in the linked data
linked_data = handle_missing_values(linked_data, trait)
print(f"Data after handling missing values: {linked_data.shape}")

# 5. Determine whether the trait and demographic features are severely biased
is_trait_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)

# 6. Conduct final quality validation and save cohort information
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=True, 
    is_trait_available=True, 
    is_biased=is_trait_biased, 
    df=linked_data,
    note="Dataset contains gene expression data from rectal cancer patients with variable responses to chemoradiation therapy."
)

# 7. If the linked data is usable, save it as a CSV file
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Data was determined to be unusable and was not saved")